La función de este notebook es usar el modelo ya entrenado en otro notebook o script, simulando una situación de producción en la se debe usar el modelo para datos nuevos, distintos a los datos del dataset de entrenamiento.

In [ ]:
import pandas as pd
import joblib

modelo = joblib.load("modelo_xgboost_fraude.pkl")
threshold = joblib.load("threshold.pkl")
columnas_modelo = joblib.load("columnas_modelo.pkl")


def predecir_fraude(path_excel, output_path=None):
    
    df = pd.read_excel(path_excel)
    
    df_original = df.copy()
    
    # Nulos
    df["K"] = df["K"].fillna(df["K"].median())
    df["C"] = df["C"].fillna(df["C"].median())
    
    # Tipos
    df['Q'] = pd.to_numeric(df['Q'], errors='coerce')
    df['R'] = pd.to_numeric(df['R'], errors='coerce')
    df['S'] = pd.to_numeric(df['S'], errors='coerce')
    df['Monto'] = pd.to_numeric(df['Monto'], errors='coerce')
    
    # One hot encoding SOLO para el modelo
    df_modelo = pd.get_dummies(df, columns=["J"], drop_first=False)
    df_modelo = df_modelo.drop(columns=["Fraude"], errors="ignore")
    df_modelo = df_modelo.reindex(columns=columnas_modelo, fill_value=0)
    
    # Predicción
    proba = modelo.predict_proba(df_modelo)[:, 1]
    pred = (proba >= threshold).astype(int)
    
    # Agregar resultados al original (sin one hot encoding)
    df_original["probabilidad_fraude"] = proba
    df_original["prediccion_fraude"] = pred
    
    if output_path:
        df_original.to_excel(output_path, index=False)
    
    return df_original

# Exportación de datos para verlos en un xlsx como los datos de entrenamiento originales.
resultado = predecir_fraude(
    "data_test.xlsx",
    output_path="predicciones.xlsx"
)

# Impresión para ver el resultado en el notebook.
resultado.head()

# En producción probablemente sería más conveniente sólo exportar el id de la transacción y si es fraude o no. 

,A,B,C,D,E,F,G,H,I,J,...,N,O,P,Q,R,S,Monto,Fraude,probabilidad_fraude,prediccion_fraude
0,0,10,50257,0,0,0,0,0,0,UY,...,1,0,5,0,0,7.25,37.51,NaN,0.989694,1
1,0,10,29014,0,0,0,0,0,0,UY,...,1,0,3,0,0,11.66,8.18,NaN,0.973853,1
2,0,7,92,0,1,0,0,0,1,UY,...,1,0,2,0,0,86.97,13.96,NaN,0.972146,1
3,9,16,50269,0,0,0,0,0,0,UY,...,1,0,5,0,0,2.51,93.67,NaN,0.990515,1
4,0,8,8180,0,0,0,0,0,0,UY,...,1,0,1,0,0,25.96,135.40,NaN,0.984206,1
